We see tons of different stores here and there through the web. Internet made it possible to trade with anyone and everywhere. We can buy goods without leaving our house, we can compare prices in different stores within seconds, we can find what we really want and do not accept just the first more or less suitable offer. And I believe it would be really interesting to look at this world through the data it produces. That's why I decided to play around with e-commerce numbers and try to understand it better.

This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. 


## Libraries and Packeges

As always, we start our analysis by setting up our environment and by importing necessary libraries.

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import random
import seaborn as sns
import math
import re
import plotly.express as px
plt.style.use('fivethirtyeight')
%matplotlib inline
from sklearn import metrics
import time, warnings
import datetime as dt
from datetime import datetime
import geopandas
from scipy import stats
from matplotlib import pylab
from collections import defaultdict
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.cluster as cluster
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from pandas.plotting import scatter_matrix
from sklearn.decomposition import PCA
from sklearn import mixture
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
import statsmodels.api as sm
from statsmodels.formula.api import ols
sns.set()
warnings.filterwarnings("ignore")

/Users/AMINO/anaconda3/envs/learn-env/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Cleaning the Data

In [2]:
# Reading the dataset

Original_df = pd.read_excel (r'Online Retail.xlsx')
Original_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
# Checking the shape

Original_df.shape

(541909, 8)

In [4]:
# Checking my column IDs

Original_df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [5]:
# How many unique values I have

Original_df.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

In [6]:
# Checking on the data type

Original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [7]:
# What are my countries 

Original_df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [8]:
# Checking on the values 

Original_df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [9]:
# Getting some more general information about the data

print("Number of transactions: ", Original_df['InvoiceNo'].nunique())
print("Number of products bought: ",Original_df['StockCode'].nunique())
print("Number of customers:", Original_df['CustomerID'].nunique() )
print("Percentage of customers NA: ", round(Original_df['CustomerID'].isnull().sum() * 100 / len(Original_df),2),"%" )
print('Number of countries: ',Original_df['Country'].nunique())

Number of transactions:  25900
Number of products bought:  4070
Number of customers: 4372
Percentage of customers NA:  24.93 %
Number of countries:  38


### Checking and dealing with null/missing values

In [10]:
# Checking for Missing Values

Original_df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

As you can see we have a lot of NaN values in our CustomerID column so we're going to remove them

In [11]:
df = Original_df[Original_df['CustomerID'].notnull()]

In [12]:
# Checking to see if NaN values were filtered out

df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

As you can see we still have null values left in the description so in our can instead deleting those rows we are going to impute them with 'UNKNOWN ITEM' at the moment 

In [13]:
df['Description'] = df['Description'].fillna('UNKNOWN ITEM')

In [14]:
# Checking to see if NaN values were filtered out

df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [15]:
# WE still have 1 missing value and we are going to clear that out

df = df[df['Description'].notnull()]

In [16]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [17]:
# Lets make all the describtion upper case so it looks more clean

df['Description'] = df['Description'].str.upper()

In [18]:
# Lets check to make sure if it worked and see what customers bought often

df['Description'].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    2070
REGENCY CAKESTAND 3 TIER              1905
JUMBO BAG RED RETROSPOT               1662
ASSORTED COLOUR BIRD ORNAMENT         1418
PARTY BUNTING                         1416
Name: Description, dtype: int64

In [19]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


### Exploring the negative Quantity and UnitePrice

In [20]:
df[df['Quantity'] < 0].head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,DISCOUNT,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
239,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,17548.0,United Kingdom
240,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
241,C536391,22553,PLASTERS IN TIN SKULLS,-24,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
939,C536506,22960,JAM MAKING SET WITH JARS,-6,2010-12-01 12:38:00,4.25,17897.0,United Kingdom


As you can see all the negative quantity starts with 'C' in InvoiceNo. The negative quantities appears to be return/canceled/discount, and maybe unknown items.

In [21]:
# For our analysis lets remove them for now

df = df[df['Quantity'] > 0]

In [22]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,397924.000000,397924.000000,397924.000000
mean,13.021823,3.116174,15294.315171
std,180.420210,22.096788,1713.169877
min,1.000000,0.000000,12346.000000
25%,2.000000,1.250000,13969.000000
50%,6.000000,1.950000,15159.000000
75%,12.000000,3.750000,16795.000000
max,80995.000000,8142.750000,18287.000000


As we can see we still have negative UnitPrice so let's filter out those as well

In [23]:
df = df[df['UnitPrice'] > 0]

In [24]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,397884.000000,397884.000000,397884.000000
mean,12.988238,3.116488,15294.423453
std,179.331775,22.097877,1713.141560
min,1.000000,0.001000,12346.000000
25%,2.000000,1.250000,13969.000000
50%,6.000000,1.950000,15159.000000
75%,12.000000,3.750000,16795.000000
max,80995.000000,8142.750000,18287.000000


In [25]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Checking for duplicates

In [26]:
# Checking for duplicates

print("Number of duplicated transactions:", len(df[df.duplicated()]))

Number of duplicated transactions: 5192


In [27]:
# Lets remove the duplicates

df.drop_duplicates(inplace = True)

In [28]:
# Checking again for duplicates

print("Number of duplicated transactions:", len(df[df.duplicated()]))

Number of duplicated transactions: 0


In [29]:
df.shape

(392692, 8)

## Featuring Dngineering & EDA

In [30]:
# Creating a another new column with the total value of each order

df['Total_Sales_Amount'] = df['Quantity'] * df['UnitPrice']

In [31]:
print(df.groupby('CustomerID')['Total_Sales_Amount'].sum().sort_values())

CustomerID
16738.0         3.75
14792.0         6.20
16454.0         6.90
17956.0        12.75
16878.0        13.30
             ...    
14911.0    143711.17
16446.0    168472.50
17450.0    194390.79
18102.0    259657.30
14646.0    280206.02
Name: Total_Sales_Amount, Length: 4338, dtype: float64


In [32]:
grouped_cleaned = df.groupby(['CustomerID', 'StockCode']).sum().reset_index()

In [33]:
grouped_purchased = grouped_cleaned.query('Quantity > 0')

In [34]:
no_products = len(grouped_purchased.StockCode.unique())
no_customers = len(grouped_purchased.CustomerID.unique())
print('Number of customers in dataset:', no_customers)
print('Number of products in dataset:', no_products)

Number of customers in dataset: 4338
Number of products in dataset: 3665


It should be taken into account that the same description could be written in different cases, let's convert all description into uppercase.

In [35]:
df['Description'] = df['Description'].str.upper()

In [36]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_Sales_Amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


Separating, month, year, weekday, and hour using datetime

In [37]:
df['Month'] = df['InvoiceDate'].dt.month
df['Year'] = df['InvoiceDate'].dt.year

In [38]:
df['WeekDay'] = df['InvoiceDate'].dt.day_name()

In [39]:
df['Hour'] = df['InvoiceDate'].dt.hour

In [40]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_Sales_Amount,Month,Year,WeekDay,Hour
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,12,2010,Wednesday,8
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,2010,Wednesday,8
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,12,2010,Wednesday,8
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,2010,Wednesday,8
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,2010,Wednesday,8


## RFM Analysis

### Recency

We want to calculate Recency to find how many days ago was the customer's last purchase

In [41]:
Recency_df= df.groupby(['CustomerID','Country'], as_index=False)['InvoiceDate'].max()

In [42]:
Recency_df.columns = ['CustomerID','Country','LastPurshaceDate']
Recency_df.head()

,CustomerID,Country,LastPurshaceDate
0,12346.0,United Kingdom,2011-01-18 10:01:00
1,12347.0,Iceland,2011-12-07 15:52:00
2,12348.0,Finland,2011-09-25 13:13:00
3,12349.0,Italy,2011-11-21 09:51:00
4,12350.0,Norway,2011-02-02 16:01:00


In [43]:
now = datetime.now()
print(now)
Recency_df['Recency'] = Recency_df['LastPurshaceDate'].apply(lambda x: (now - x).days)

2020-12-17 19:48:52.949423


In [44]:
Recency_df.head()

,CustomerID,Country,LastPurshaceDate,Recency
0,12346.0,United Kingdom,2011-01-18 10:01:00,3621
1,12347.0,Iceland,2011-12-07 15:52:00,3298
2,12348.0,Finland,2011-09-25 13:13:00,3371
3,12349.0,Italy,2011-11-21 09:51:00,3314
4,12350.0,Norway,2011-02-02 16:01:00,3606


### Frequency

How many times the customer made a purchase

In [45]:
Frequency_df = df.groupby(by=['CustomerID','Country'], as_index=False)['InvoiceNo'].count()
Frequency_df.columns = ['CustomerID','Country','Frequency']
Frequency_df.head()

,CustomerID,Country,Frequency
0,12346.0,United Kingdom,1
1,12347.0,Iceland,182
2,12348.0,Finland,31
3,12349.0,Italy,73
4,12350.0,Norway,17


### Monetary

We want to know how much the customer spent over time

In [46]:
Monetary_df = df.groupby(by=['CustomerID','Country'], as_index=False).agg({'Total_Sales_Amount': 'sum'})

In [47]:
Monetary_df.columns = ['CustomerID','Country','Monetary']
Monetary_df.head()

,CustomerID,Country,Monetary
0,12346.0,United Kingdom,77183.60
1,12347.0,Iceland,4310.00
2,12348.0,Finland,1797.24
3,12349.0,Italy,1757.55
4,12350.0,Norway,334.40


Now lets create a RFM table

In [48]:
RFM_Table = Recency_df.merge(Frequency_df,on=['CustomerID','Country'])

In [49]:
RFM_df = RFM_Table.merge(Monetary_df,on=['CustomerID','Country'])
RFM_df.head()

,CustomerID,Country,LastPurshaceDate,Recency,Frequency,Monetary
0,12346.0,United Kingdom,2011-01-18 10:01:00,3621,1,77183.60
1,12347.0,Iceland,2011-12-07 15:52:00,3298,182,4310.00
2,12348.0,Finland,2011-09-25 13:13:00,3371,31,1797.24
3,12349.0,Italy,2011-11-21 09:51:00,3314,73,1757.55
4,12350.0,Norway,2011-02-02 16:01:00,3606,17,334.40


In [50]:
RFM_df.set_index(['CustomerID','Country'],inplace=True)

In [51]:
RFM_df.head()

,,LastPurshaceDate,Recency,Frequency,Monetary
CustomerID,Country,,,,
12346.0,United Kingdom,2011-01-18 10:01:00,3621,1,77183.60
12347.0,Iceland,2011-12-07 15:52:00,3298,182,4310.00
12348.0,Finland,2011-09-25 13:13:00,3371,31,1797.24
12349.0,Italy,2011-11-21 09:51:00,3314,73,1757.55
12350.0,Norway,2011-02-02 16:01:00,3606,17,334.40


Let's see the application of Pareto Principle. It is commonly referred to as the 80-20 rule on our dataset by applying it to our RFM variables.

In [52]:
pareto_cutoff = RFM_df['Monetary'].sum() * 0.8
print("The 80% of total revenue is: ",round(pareto_cutoff,2))

The 80% of total revenue is:  7109767.12


In [53]:
CR = RFM_df

We're going to create a rank column and rank our top customers

In [54]:
CR['Rank'] = CR['Monetary'].rank(ascending=0)
CR.head()

,,LastPurshaceDate,Recency,Frequency,Monetary,Rank
CustomerID,Country,,,,,
12346.0,United Kingdom,2011-01-18 10:01:00,3621,1,77183.60,10.0
12347.0,Iceland,2011-12-07 15:52:00,3298,182,4310.00,335.0
12348.0,Finland,2011-09-25 13:13:00,3371,31,1797.24,1003.0
12349.0,Italy,2011-11-21 09:51:00,3314,73,1757.55,1026.0
12350.0,Norway,2011-02-02 16:01:00,3606,17,334.40,3104.0


In [55]:
CR.sort_values('Rank',ascending=True)

,,LastPurshaceDate,Recency,Frequency,Monetary,Rank
CustomerID,Country,,,,,
14646.0,Netherlands,2011-12-08 12:12:00,3297,2076,280206.02,1.0
18102.0,United Kingdom,2011-12-09 11:50:00,3296,431,259657.30,2.0
17450.0,United Kingdom,2011-12-01 13:29:00,3304,336,194390.79,3.0
16446.0,United Kingdom,2011-12-09 09:15:00,3296,3,168472.50,4.0
14911.0,EIRE,2011-12-08 15:54:00,3297,5670,143711.17,5.0
...,...,...,...,...,...,...
16878.0,United Kingdom,2011-09-16 17:39:00,3380,3,13.30,4342.0
17956.0,United Kingdom,2011-04-04 13:47:00,3545,1,12.75,4343.0
16454.0,United Kingdom,2011-10-26 11:40:00,3340,2,6.90,4344.0


We want to see the top 20% of our customers

In [56]:
Top_20_Customers  = 4346 *20 /100
Top_20_Customers

869.2

This will show how much revenue our top 20% customer generates

In [57]:
Top_20_Revenue = CR[CR['Rank'] <= 867]['Monetary'].sum()
Top_20_Revenue

6632400.831

For our future study we can concentrate on those top 20% customers since they generate most of the revenue. Pareto’s rule says 80% of the results come from 20% of the causes.

Using Quartiles to create customers segments from model

In [58]:
Quantiles = RFM_df.quantile(q=[0.25,0.5,0.75])
Quantiles

,Recency,Frequency,Monetary,Rank
0.25,3313.0,17.0,306.5925,1087.25
0.50,3346.0,41.0,666.8750,2173.50
0.75,3438.0,98.0,1656.5375,3259.75


We are going to right a function to put our customers in different quartiles

In [59]:
def Recency_Score(v,rfm,q):
    if v <= q[rfm][0.25]:
        return 4
    elif v <= q[rfm][0.50]:
        return 3
    elif v <= q[rfm][0.75]: 
        return 2
    else:
        return 1

In [60]:
def Frequency_Monetary_Score(v,rfm,q):
    if v <= q[rfm][0.25]:
        return 1
    elif v <= q[rfm][0.50]:
        return 2
    elif v <= q[rfm][0.75]: 
        return 3
    else:
        return 4

Now we are going to add them to our dataset using RFM and our function

In [61]:
RFM_Segmentation = RFM_df
RFM_Segmentation['R_Quartile'] = RFM_Segmentation['Recency'].apply(Recency_Score, args=('Recency',Quantiles,))
RFM_Segmentation['F_Quartile'] = RFM_Segmentation['Frequency'].apply(Frequency_Monetary_Score, args=('Frequency',Quantiles,))
RFM_Segmentation['M_Quartile'] = RFM_Segmentation['Monetary'].apply(Frequency_Monetary_Score, args=('Monetary',Quantiles,))

We are going to drop LastPurshaceDate since we don't need it anymore

In [62]:
RFM_Segmentation.drop('LastPurshaceDate',axis=1,inplace=True)

Best Recency score = 4: Most recently purchase.

Best Frequency score = 4: Most quantity purchase.

Best Monetary score = 4: Spent the most.

In [63]:
RFM_Segmentation.head()

,,Recency,Frequency,Monetary,Rank,R_Quartile,F_Quartile,M_Quartile
CustomerID,Country,,,,,,,
12346.0,United Kingdom,3621,1,77183.60,10.0,1,1,4
12347.0,Iceland,3298,182,4310.00,335.0,4,4,4
12348.0,Finland,3371,31,1797.24,1003.0,2,2,4
12349.0,Italy,3314,73,1757.55,1026.0,3,3,4
12350.0,Norway,3606,17,334.40,3104.0,1,1,2


Now lets add all the RFM column and create a RFM score

In [64]:
RFM_Segmentation['RFM_Score'] = RFM_Segmentation.R_Quartile.map(str) \
                            + RFM_Segmentation.F_Quartile.map(str) \
                            + RFM_Segmentation.M_Quartile.map(str)
RFM_Segmentation.head()

,,Recency,Frequency,Monetary,Rank,R_Quartile,F_Quartile,M_Quartile,RFM_Score
CustomerID,Country,,,,,,,,
12346.0,United Kingdom,3621,1,77183.60,10.0,1,1,4,114
12347.0,Iceland,3298,182,4310.00,335.0,4,4,4,444
12348.0,Finland,3371,31,1797.24,1003.0,2,2,4,224
12349.0,Italy,3314,73,1757.55,1026.0,3,3,4,334
12350.0,Norway,3606,17,334.40,3104.0,1,1,2,112


Let's get a sample to make sure everything is correct

In [65]:
RFM_Segmentation.sample(10)

,,Recency,Frequency,Monetary,Rank,R_Quartile,F_Quartile,M_Quartile,RFM_Score
CustomerID,Country,,,,,,,,
12837.0,United Kingdom,3469,12,134.10,4025.0,1,1,1,111
14710.0,United Kingdom,3310,136,599.48,2344.0,4,4,2,442
16257.0,United Kingdom,3472,2,46.95,4312.0,1,1,1,111
15655.0,United Kingdom,3539,34,560.67,2433.0,1,2,2,122
17566.0,United Kingdom,3303,142,869.77,1823.0,4,4,3,443
17118.0,United Kingdom,3613,10,157.02,3902.0,1,1,1,111
14083.0,United Kingdom,3300,177,1002.90,1659.0,4,4,3,443
15746.0,United Kingdom,3430,28,196.88,3710.0,2,2,1,221
15670.0,United Kingdom,3592,12,173.05,3819.0,1,1,1,111


In [66]:
RFM_Segmentation[RFM_Segmentation['RFM_Score']=='444'].sort_values('Monetary', ascending=False).head(10)


,,Recency,Frequency,Monetary,Rank,R_Quartile,F_Quartile,M_Quartile,RFM_Score
CustomerID,Country,,,,,,,,
14646.0,Netherlands,3297,2076,280206.02,1.0,4,4,4,444
18102.0,United Kingdom,3296,431,259657.30,2.0,4,4,4,444
17450.0,United Kingdom,3304,336,194390.79,3.0,4,4,4,444
14911.0,EIRE,3297,5670,143711.17,5.0,4,4,4,444
14156.0,EIRE,3305,1395,117210.08,7.0,4,4,4,444
17511.0,United Kingdom,3298,963,91062.38,8.0,4,4,4,444
16684.0,United Kingdom,3300,277,66653.56,11.0,4,4,4,444
14096.0,United Kingdom,3300,5111,65164.79,12.0,4,4,4,444
13694.0,United Kingdom,3299,568,65039.62,13.0,4,4,4,444


I want to see how many customers I have in different Segmentation categories 

In [67]:
print("Best Customers: ",len(RFM_Segmentation[RFM_Segmentation['RFM_Score']=='444']))
print('Loyal Customers: ',len(RFM_Segmentation[RFM_Segmentation['F_Quartile']==4]))
print("Big Spenders: ",len(RFM_Segmentation[RFM_Segmentation['M_Quartile']==4]))
print('Almost Lost: ', len(RFM_Segmentation[RFM_Segmentation['RFM_Score']=='244']))
print('Recent Customers: ', len(RFM_Segmentation[RFM_Segmentation['RFM_Score']=='443']))
print('Lost Customers: ',len(RFM_Segmentation[RFM_Segmentation['RFM_Score']=='144']))
print('Lost Cheap Customers: ',len(RFM_Segmentation[RFM_Segmentation['RFM_Score']=='111']))

Best Customers:  443
Loyal Customers:  1080
Big Spenders:  1087
Almost Lost:  105
Recent Customers:  87
Lost Customers:  22
Lost Cheap Customers:  383


I want create another column that describe the customers segments category based on their RFM score and fill the rest with 'Others' for now, so this way we can easily interpret the RFM score

In [68]:
Segment_Class = {'Best Customers': '444',
'Loyal Customers': '344',
'Big Spenders': '334',
'Almost Lost': '244',
'Recent Customers': '443',
'Lost Customers': '144',
'Lost Cheap Customers': '122'}

In [69]:
Class_Segments = dict(zip(Segment_Class.values(),Segment_Class.keys()))

In [70]:
RFM_df['Customers Segments'] = RFM_df.RFM_Score.map(lambda x: Class_Segments.get(x))

In [71]:
RFM_df.fillna('Others', inplace=True)

In [72]:
RFM_df.sample(10)

,,Recency,Frequency,Monetary,Rank,R_Quartile,F_Quartile,M_Quartile,RFM_Score,Customers Segments
CustomerID,Country,,,,,,,,,
15637.0,United Kingdom,3348,15,223.36,3584.5,2,1,1,211,Others
13767.0,United Kingdom,3298,368,17220.36,52.0,4,4,4,444,Best Customers
17656.0,United Kingdom,3369,82,1674.69,1077.0,2,3,4,234,Others
17775.0,United Kingdom,3550,3,56.40,4297.0,1,1,1,111,Others
13577.0,United Kingdom,3321,83,1687.60,1072.0,3,3,4,334,Big Spenders
17162.0,United Kingdom,3324,104,1715.46,1046.0,3,4,4,344,Loyal Customers
17503.0,United Kingdom,3298,38,558.96,2440.0,4,2,2,422,Others
17468.0,United Kingdom,3306,5,137.00,4007.0,4,1,1,411,Others
13742.0,United Kingdom,3312,313,3107.87,528.0,4,4,4,444,Best Customers
